In [ ]:
import gnssmapper as gm
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
mapfile='../maps/tcr/tcr.gpkg'
obsfile='../data/tcr.csv'


In [ ]:
map_ = gpd.read_file(mapfile)
height = pd.read_csv(
    '../maps/tq/tq.csv', names=pd.read_csv('../maps/tcr/docs/BHA_Header.csv').columns)
map_ = map_.merge(height, left_on='fid', right_on='OS_TOPO_TOID')
map_['height'] = (map_.RelH2+map_.RelHMax)/2
map_ = map_[['height', 'geometry']]
map_ = map_.set_crs(7405, allow_override=True)
gm.geo.pyproj.network.set_network_enabled()


In [ ]:
obs=pd.read_csv(obsfile,index_col=0)



In [ ]:
obs

In [ ]:
xy=obs[['x','y']].drop_duplicates()
points=gpd.GeoSeries(gpd.points_from_xy(xy.x,xy.y,crs=7405))

In [ ]:
points.shape

In [ ]:
base=map_.plot(column='height',legend=True)
points.plot(ax=base,marker='.',color='red',markersize=1,alpha=0.3)

In [ ]:
from metrics import int_metric

In [ ]:
map_['with_occluded_I_inf']=[int_metric(obs[str(i)],h,np.inf) for i,h in enumerate(map_.height)]
map_['with_occluded_I_8']=[int_metric(obs[str(i)],h,8) for i,h in enumerate(map_.height)]
map_['with_occluded_I_2']=[int_metric(obs[str(i)],h,2) for i,h in enumerate(map_.height)]
obs2=obs.drop(columns=['x','y','t'])
obs2[~obs2.apply(lambda x: x<x.min()+1e-6,axis=1)]=np.nan
map_['I_inf']=[int_metric(obs2[str(i)],h,np.inf) for i,h in enumerate(map_.height)]
map_['I_8']=[int_metric(obs2[str(i)],h,8) for i,h in enumerate(map_.height)]
map_['I_2']=[int_metric(obs2[str(i)],h,2) for i,h in enumerate(map_.height)]


In [ ]:
f,axes=plt.subplots(2,2,figsize=(8,8))
map_.plot(ax=axes[0,0],column='with_occluded_I_inf',legend=True)
map_.plot(ax=axes[0,1],column='with_occluded_I_2',legend=True)
map_.plot(ax=axes[1,0],column='I_inf',legend=True)
map_.plot(ax=axes[1,1],column='I_2',legend=True)
plt.tight_layout()
plt.show()